## Pose Estimation Walkthrough

this notebook contains a note while toying with data and the result, also the steps to produce the result

In [1]:
import torch
import syft as sy
import numpy as np

LABEL = ['Standing still', 'Sitting and relaxing', 'Lying down', 'Walking', 'Climbing', 'Running']
N_WORKER = 8
BATCH_SIZE = 20
VALID_SIZE = 0.1

In [2]:
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities to support Federated Learning

def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]

workers = connect_to_workers(n_workers = N_WORKER)

## Reading data

1. Split files detected by glob to train and test
2. Read CSV
3. Construct Dataset class

For the first experiment, i will try to use all features that available so the input will be [1 x 21]

In [3]:
import glob
import csv
import random
import math

# folder path
PATH = 'data/Preprocessed'
TRAIN_FILES_COUNT = 8
# seed for random
random.seed(300)

In [4]:
files = glob.glob(PATH+"/*.csv")

# split into train and test
# according to note, only 8 data for training, and the rest for testing
count_valid = math.ceil(VALID_SIZE*TRAIN_FILES_COUNT)
random.shuffle(files)
valid_files,train_files, test_files = files[0:count_valid],files[count_valid:TRAIN_FILES_COUNT],files[TRAIN_FILES_COUNT:]

In [5]:
index_to_read = 3
sample_read = []
sample_label = []
label = []

with open(train_files[index_to_read]) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if int(row[21]) != 0:
            if int(row[21]) not in label:
                label.append(int(row[21]))
            sample_read.append([float(item) for item in row[0:21]])
            sample_label.append(int(row[21]))

print(train_files[index_to_read])
print('types of label inside: {}'.format(label))

data/Preprocessed\mHealth_subject2.csv
types of label inside: [5, 1, 2, 3, 4, 6]


In [6]:
# print sample data
for index,rep in enumerate(sample_read[:3]):
    print('Chest \t\t\tLeft Angkle')
    print('{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f}'.format(*rep))
    print('Label \t\t\tRight Angkle')
    print('{} \t\t\t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f} \t{:.2f}'.format(label[index],*rep[12:]))
    print('')

Chest 			Left Angkle
-9.15 	0.64 	0.90 	0.18 	-9.49 	-6.48 	-0.27 	-0.39 	0.17 	-94.93 	-29.88 	11.43
Label 			Right Angkle
5 			-3.45 	-10.68 	2.34 	-0.41 	-0.40 	0.01 	5.76 	19.53 	-1.16

Chest 			Left Angkle
-8.62 	0.36 	1.05 	0.82 	-7.38 	-3.79 	-0.34 	-0.50 	0.01 	-80.26 	-37.18 	9.77
Label 			Right Angkle
1 			-3.69 	-10.95 	1.98 	-0.37 	-0.41 	-0.05 	7.69 	14.47 	20.59

Chest 			Left Angkle
-7.90 	0.03 	0.67 	0.72 	-4.94 	-1.20 	-0.34 	-0.50 	0.01 	-61.56 	-19.20 	6.19
Label 			Right Angkle
2 			-3.59 	-10.21 	2.15 	-0.37 	-0.41 	-0.05 	10.89 	11.56 	41.26



### Use Constructed Dataset

To make it easy, we create a dataloader for reading and returning value from csv files

In [8]:
from dataloader import ImuPoseDataset

train_dataset = ImuPoseDataset(files=train_files)
valid_dataset = ImuPoseDataset(files=valid_files)
test_dataset = ImuPoseDataset(files=test_files)

print("train dataset: {}".format(len(train_dataset)))
print("valid dataset: {}".format(len(valid_dataset)))
print("test dataset: {}".format(len(test_dataset)))

train dataset: 129024
valid dataset: 18432
test dataset: 36864


### Construct Dataloader

To iterate our dataset we will use Pytorch built-in dataloader. Since we are training with federated learning we will need to transform it into federatedDataset

In [9]:
federated_train_loader = sy.FederatedDataLoader( train_dataset
                                                .federate(workers), # <-- we distribute the dataset across all the workers, it's now a FederatedDataset
                                                batch_size=BATCH_SIZE)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=BATCH_SIZE)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=1)